# Deploying Machine Learning Models using kubectl
This demo shows how you can interact directly with kubernetes using kubectl to create and manage runtime machine learning models. It uses Minikube as the target Kubernetes cluster.
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)

Start minikube and ensure custom resource validation is activated and ther is 5G of memory.

In [ ]:
!minikube start --memory=5000 --feature-gates=CustomResourceValidation=true

Install Helm

In [1]:
!helm init

Creating /home/clive/.helm 
Creating /home/clive/.helm/repository 
Creating /home/clive/.helm/repository/cache 
Creating /home/clive/.helm/repository/local 
Creating /home/clive/.helm/plugins 
Creating /home/clive/.helm/starters 
Creating /home/clive/.helm/cache/archive 
Creating /home/clive/.helm/repository/repositories.yaml 
Adding stable repo with URL: https://kubernetes-charts.storage.googleapis.com 
Adding local repo with URL: http://127.0.0.1:8879/charts 
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.
Happy Helming!


Label the node to allow load testing to run on it

In [32]:
!kubectl label nodes `kubectl get nodes -o jsonpath='{.items[0].metadata.name}'` role=locust --overwrite

node "minikube" labeled


## Start seldon-core

Install the custom resource definition

In [7]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd

NAME:   seldon-core-crd
LAST DEPLOYED: Sat Feb 17 09:48:19 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s


NOTES:
NOTES: TODO




In [9]:
!kubectl create namespace seldon

namespace "seldon" created


In [10]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon

NAME:   seldon-core
LAST DEPLOYED: Sat Feb 17 09:48:57 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                    DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-apiserver        1        1        1           0          0s
seldon-cluster-manager  1        0        0           0          0s
redis                   1        0        0           0          0s

==> v1/Service
NAME              TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)                        AGE
seldon-apiserver  NodePort   10.108.253.147  <none>       8080:30370/TCP,5000:31436/TCP  0s
redis             ClusterIP  10.102.153.105  <none>       6379/TCP                       0s

==> v1/Pod(related)
NAME                                    READY  STATUS             RESTARTS  AGE
seldon-apiserver-5596577b55-d29zg       0/1    ContainerCreating  0         0s
seldon-cluster-manager-576c6ddf5-b7s6w  0/1    ContainerCreating  0         0s
redis-df886d999-6fvfx                   0/1  

Install prometheus and grafana for analytics

In [11]:
!helm install ../helm-charts/seldon-core-analytics --name seldon-core-analytics \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --namespace seldon

NAME:   seldon-core-analytics
LAST DEPLOYED: Sat Feb 17 09:51:20 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/Secret
NAME                 TYPE    DATA  AGE
grafana-prom-secret  Opaque  1     1s

==> v1/ConfigMap
NAME                       DATA  AGE
alertmanager-server-conf   1     1s
grafana-import-dashboards  5     1s
prometheus-rules           4     1s
prometheus-server-conf     1     1s

==> v1/Job
NAME                            DESIRED  SUCCESSFUL  AGE
grafana-prom-import-dashboards  1        0           1s

==> v1beta1/Deployment
NAME                     DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
alertmanager-deployment  1        1        1           0          1s
grafana-prom-deployment  1        1        1           0          1s
prometheus-deployment    1        1        1           0          1s

==> v1/Service
NAME                      TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)       AGE
alertmanager              ClusterIP  10.110.34.128  <none>       80/

Check all services are running before proceeding.

In [13]:
!kubectl get pods -n seldon

NAME                                       READY     STATUS    RESTARTS   AGE
alertmanager-deployment-7fbfdfdfb6-lpqrm   1/1       Running   0          1m
grafana-prom-deployment-7b45fb85d4-prswq   1/1       Running   0          1m
prometheus-deployment-5749d9db5-bmb5n      1/1       Running   0          1m
prometheus-node-exporter-smq4c             1/1       Running   0          1m
redis-df886d999-6fvfx                      1/1       Running   0          4m
seldon-apiserver-5596577b55-d29zg          1/1       Running   0          4m
seldon-cluster-manager-576c6ddf5-b7s6w     1/1       Running   0          4m


## Set up REST and gRPC methods

Install gRPC modules for the prediction protos.

In [14]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I./proto --python_out=./proto --grpc_python_out=./proto ./proto/prediction.proto

Illustration of both REST and gRPC requests. 

In [15]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
import commands

NAMESPACE='seldon'
MINIKUBE_IP=commands.getoutput('minikube ip')
MINIKUBE_HTTP_PORT=commands.getoutput("kubectl get svc -n "+NAMESPACE+" -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'")
MINIKUBE_GRPC_PORT=commands.getoutput("kubectl get svc -n "+NAMESPACE+" -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[1].nodePort}'")

def get_token():
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://"+MINIKUBE_IP+":"+MINIKUBE_HTTP_PORT+"/oauth/token",
                auth=HTTPBasicAuth('oauth-key', 'oauth-secret'),
                data=payload)
    print response.text
    token =  response.json()["access_token"]
    return token

def rest_request():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://"+MINIKUBE_IP+":"+MINIKUBE_HTTP_PORT+"/api/v0.1/predictions",
                headers=headers,
                json=payload)
    print response.text
    
def grpc_request():
    token = get_token()
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel(MINIKUBE_IP+":"+MINIKUBE_GRPC_PORT)
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print response


# Integrating with Kubernetes API

## Validation

Using OpenAPI Schema certain basic validation can be done before the custom resource is accepted.

In [16]:
!kubectl create -f resources/model_invalid1.json

The SeldonDeployment "seldon-deployment-example" is invalid: []: Invalid value: map[string]interface {}{"apiVersion":"machinelearning.seldon.io/v1alpha1", "kind":"SeldonDeployment", "metadata":map[string]interface {}{"labels":map[string]interface {}{"app":"seldon"}, "name":"seldon-deployment-example", "namespace":"seldon", "creationTimestamp":"2018-02-17T09:53:36Z", "uid":"6c135aca-13c8-11e8-93d3-0800277d1140", "selfLink":"", "clusterName":""}, "spec":map[string]interface {}{"predictors":[]interface {}{map[string]interface {}{"annotations":map[string]interface {}{"predictor_version":"v1"}, "componentSpec":map[string]interface {}{"spec":map[string]interface {}{"containers":[]interface {}{map[string]interface {}{"image":"seldonio/mean_classifier:0.6", "imagePullPolicy":22, "name":"mean-classifier", "resources":map[string]interface {}{"requests":map[string]interface {}{"memory":"1Mi"}}}}, "terminationGracePeriodSeconds":20}}, "graph":map[string]interface {}{"children":[]interface {}{}, "e

## Normal Operation
A simple example is shown below we use a single prepacked model for illustration. The spec contains a set of predictors each of which contains a ***componentSpec*** which is a Kubernetes [PodTemplateSpec](https://kubernetes.io/docs/api-reference/v1.9/#podtemplatespec-v1-core) alongside a ***graph*** which describes how components fit together.

In [18]:
!pygmentize resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example",
	"namespace": "seldon"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction",
            "deployment_version": "v1"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpec": {
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "mem

## Create Seldon Deployment

Deploy the runtime graph to kubernetes.

In [19]:
!kubectl apply -f resources/model.json

seldondeployment "seldon-deployment-example" created


In [20]:
!kubectl get seldondeployments -n seldon

NAME                        AGE
seldon-deployment-example   9s


In [21]:
!kubectl describe seldondeployments seldon-deployment-example -n seldon

Name:         seldon-deployment-example
Namespace:    seldon
Labels:       app=seldon
Annotations:  kubectl.kubernetes.io/last-applied-configuration={"apiVersion":"machinelearning.seldon.io/v1alpha1","kind":"SeldonDeployment","metadata":{"name":"seldon-deployment-example","namespace":"seldon","selfL...
API Version:  machinelearning.seldon.io/v1alpha1
Kind:         SeldonDeployment
Metadata:
  Cluster Name:        
  Creation Timestamp:  2018-02-17T09:54:46Z
  Generation:          0
  Resource Version:    1871
  Self Link:           /apis/machinelearning.seldon.io/v1alpha1/namespaces/seldon/seldondeployments/seldon-deployment-example
  UID:                 960dabba-13c8-11e8-93d3-0800277d1140
Spec:
  Annotations:
    Deployment _ Version:  v1
    Project _ Name:        FX Market Prediction
  Name:                    test-deployment
  Oauth _ Key:             oauth-key
  Oauth _ Secret:          oauth-secret
  Predictors:
    Annotations:
      Predictor _ Version:  v1
    Component Spec

Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [22]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:test-deployment-fx-market-predictor replicas:1 replicasAvailable:1]]]

## Get predictions

#### REST Request

In [23]:
rest_request()

{"access_token":"28de63ce-0001-47a8-ad76-54e00f6ad300","token_type":"bearer","expires_in":43199,"scope":"read write"}
{
  "meta": {
    "puid": "9uhc299436fhfa4tdpa8l8gnph",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.05133579311531625, 0.12823373759251927]
    }
  }
}


#### gRPC Request

In [24]:
grpc_request()

{"access_token":"28de63ce-0001-47a8-ad76-54e00f6ad300","token_type":"bearer","expires_in":43198,"scope":"read write"}
meta {
  puid: "o3l8k9c0up05vu5g8orshpp7pd"
}
data {
  names: "proba"
  tensor {
    shape: 3
    shape: 1
    values: 0.128233737593
    values: 0.397314662022
    values: 0.829676081356
  }
}



## Update deployment with canary

We will change the deployment to add a "canary" deployment. This illustrates:
 - Updating a deployment with no downtime
 - Adding an extra predictor to run alongside th exsting predictor.
 
 You could manage different traffic levels by controlling the number of replicas of each.

In [26]:
!pygmentize resources/model_with_canary.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example",
	"namespace" : "seldon"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpec": {
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mean_classifier:0.6",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "mean-classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                      

In [27]:
!kubectl apply -f resources/model_with_canary.json

seldondeployment "seldon-deployment-example" configured


Check the status of the deployments. Note: **Might need to run several times until replicasAvailable is 1 for both predictors**.

In [28]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:test-deployment-fx-market-predictor replicas:1 replicasAvailable:1] map[replicasAvailable:1 name:test-deployment-fx-market-predictor-canary replicas:1]]]

#### REST Request

In [29]:
rest_request()

{"access_token":"28de63ce-0001-47a8-ad76-54e00f6ad300","token_type":"bearer","expires_in":43006,"scope":"read write"}
{
  "meta": {
    "puid": "tjsp6p2etelhl78a0b5ajuheq6",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.05133579311531625, 0.12823373759251927]
    }
  }
}


#### gRPC request

In [30]:
grpc_request()

{"access_token":"28de63ce-0001-47a8-ad76-54e00f6ad300","token_type":"bearer","expires_in":43002,"scope":"read write"}
meta {
  puid: "8g0nuebc55mh5f3gm55hj3pfmu"
}
data {
  names: "proba"
  tensor {
    shape: 3
    shape: 1
    values: 0.128233737593
    values: 0.397314662022
    values: 0.829676081356
  }
}



## Load test

Start a load test which will post REST requests at 10 requests per second.

In [31]:
!helm install seldon-core-loadtesting --name loadtest  \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --namespace seldon \
    --repo https://storage.googleapis.com/seldon-charts

NAME:   loadtest
LAST DEPLOYED: Sat Feb 17 10:03:54 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ReplicationController
NAME             DESIRED  CURRENT  READY  AGE
locust-slave-1   1        1        0      0s
locust-master-1  1        1        0      0s

==> v1/Service
NAME             TYPE      CLUSTER-IP      EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.101.224.162  <none>       5557:30526/TCP,5558:30920/TCP,8089:30915/TCP  0s

==> v1/Pod(related)
NAME                   READY  STATUS   RESTARTS  AGE
locust-slave-1-ppx8s   0/1    Pending  0         0s
locust-master-1-pvsr2  0/1    Pending  0         0s




You should port-foward the grafana dashboard

```bash
kubectl port-forward $(kubectl get pods -n seldon -l app=grafana-prom-server -o jsonpath='{.items[0].metadata.name}') -n seldon 3000:3000
```

You can then iew an analytics dashboard inside the cluster at http://localhost:3000/dashboard/db/prediction-analytics?refresh=5s&orgId=1. Your IP address may be different. get it via minikube ip. Login with:
 - Username : admin
 - password : password (as set when starting seldon-core above)
 
 The dashboard should look like below:
 
 
 <img src="images/dashboard.png" alt="predictor with canary" title="ml graph"/>

# Tear down

In [33]:
!helm delete loadtest --purge

release "loadtest" deleted


In [34]:
!kubectl delete -f resources/model_with_canary.json

seldondeployment "seldon-deployment-example" deleted


In [35]:
!helm delete seldon-core-analytics --purge

release "seldon-core-analytics" deleted


In [36]:
!helm delete seldon-core --purge

release "seldon-core" deleted


In [37]:
!helm delete seldon-core-crd --purge

release "seldon-core-crd" deleted
